## LightFM

A hybrid latent representation recommender model.

The model learns embeddings (latent representations in a high-dimensional space) for users and items in a way that encodes user preferences over items. When multiplied together, these representations produce scores for every item for a given user; items scored highly are more likely to be interesting to the user.

### Python Packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from lightfm import LightFM
from lightfm.data import Dataset
from scipy.sparse import coo_matrix
from lightfm.evaluation import precision_at_k,recall_at_k,auc_score,reciprocal_rank
from lightfm.cross_validation import random_train_test_split

### Data Source

In [3]:
#Products Dataset
asin_product_mapping = pd.read_csv('dataset/asin_product_mapping.csv')

#Item Categories
itemset_preprocessed = pd.read_csv('dataset/itemset_preprocessed.csv')

#Utility
df_utility = pd.read_csv('dataset/utility_topn.csv')
reviews.drop(['Unnamed: 0'], axis=1, inplace=True)

#Main Dataset
reviews = pd.read_csv('dataset/reviews.csv')
reviews.drop(['Unnamed: 0'], axis=1, inplace=True)
reviews.head()

,ASIN,ProductName,reviewerID,reviewRating,reviewDate,reviewLocation,reviewVotes
0,b001f30182,star_wars_the_black_series_dark_trooper_toy_6i...,TK-815TK-815_AH4NM3KGMDME7SKRSJD73TIBABIQ,5.0,"September 21, 2022",United States,13
1,b001f30182,star_wars_the_black_series_dark_trooper_toy_6i...,vyeranos_AESWEZHPRRYTZRW2FBIJSXEKGTFA,5.0,"January 13, 2024",United States,0
2,b001f30182,star_wars_the_black_series_dark_trooper_toy_6i...,Marty_AHGIPPEB6HKSXNPB6Y3DLBKCVRHQ,5.0,"November 24, 2023",United States,0
3,b001f30182,star_wars_the_black_series_dark_trooper_toy_6i...,Kati_AHWQ73WR7MVGBKDB5YE6C4WDPNJA,5.0,"January 2, 2024",United States,0
4,b001f30182,star_wars_the_black_series_dark_trooper_toy_6i...,Lochlan LongstriderLochlan Longstrider_AHSHUIZ...,4.0,"September 18, 2023",United States,0


In [4]:
reviews['reviewDate'] = pd.to_datetime(reviews['reviewDate'])

In [5]:
# Remove Duplicate Accounts
reviewers_to_remove = [
    'Amazon CustomerAmazon Customer_',
    'Amazon Customer_',
    'Cliente de Amazon_',
    'Cliente Amazon_',
    'Kindle Customer_',
    'Client d\'Amazon_',
    'Amazon Customer',
    'Amazon Customer_',
    'Amazon Kunde_',
    'Amazon カスタマー_',
    'Cliente Kindle_',
    'Cliente de Kindle_'
]

reviews = reviews[~reviews['reviewerID'].isin(reviewers_to_remove)]

### Build Dataset Requirement for LightFM

Tool for building interaction and feature matrices, taking care of the mapping between user/item ids and feature names and internal feature indices.

In [6]:
# Create a dataset object
dataset = Dataset()
dataset.fit(users=reviews['reviewerID'].unique(),
            items=reviews['ASIN'].unique())

# Build the interactions matrix
(interactions, _) = dataset.build_interactions(zip(reviews['reviewerID'], 
                                                   reviews['ASIN']))
                                                   #reviews['reviewRating']))

Call build_interactions with an iterable of (user id, item id) or (user id, item id, weight) to build an interactions and weights matrix

### LightFM Training Model

- Logistic: useful when both positive (1) and negative (-1) interactions are present.
- BPR: Bayesian Personalised Ranking 1 pairwise loss. Maximises the prediction difference between a positive example and a randomly chosen negative example. Useful when only positive interactions are present and optimising ROC AUC is desired.
- WARP: Weighted Approximate-Rank Pairwise 2 loss. Maximises the rank of positive examples by repeatedly sampling negative examples until rank violating one is found. Useful when only positive interactions are present and optimising the top of the recommendation list (precision@k) is desired.
- k-OS WARP: k-th order statistic loss 3. A modification of WARP that uses the k-th positive example for any given user as a basis for pairwise updates.

In [7]:
Train, Test = random_train_test_split(interactions, test_percentage=0.3, random_state=42)

model = LightFM(loss='warp', no_components=5, random_state=42)
                #learning_rate=0.1,
                #no_components=5,
                #item_alpha=0.01,
                #user_alpha=0.01)

model.fit(Train, epochs=30, num_threads=2)

### Model Recommendations (Predictions)

In [8]:
reviews['reviewerID'].value_counts().head(10)

reviewerID
Chris_     67
David_     65
Laura_     62
Maria_     61
Sarah_     60
Alex_      58
Andrea_    50
Daniel_    50
Carlos_    46
John_      45
Name: count, dtype: int64

In [9]:
def get_top_n_recommendations(model, dataset, user_id, n=10):
    # Map the user ID to the internal user index in LightFM
    user_index = dataset.mapping()[0][user_id]
    
    # Number of items in the dataset
    n_items = dataset.interactions_shape()[1]
    
    # Generate predictions for all items for this user
    scores = model.predict(user_index, np.arange(n_items))
    
    # Rank items by score in descending order
    top_items_indices = np.argsort(-scores)[:n]
    
    # Map the item indices back to their original IDs
    reverse_item_map = {v: k for k, v in dataset.mapping()[2].items()}
    top_item_ids = [reverse_item_map[i] for i in top_items_indices]
    
    return top_item_ids

In [10]:
def get_top_n_recommendations_with_latest_transactions(model, dataset, user_id, transactions_df, n=10):

    # Generate predictions for all items for this user
    user_index = dataset.mapping()[0][user_id]
    
    # Retrieve the actual item indices from the dataset mapping
    item_indices = list(dataset.mapping()[2].values())  # This ensures you're using the correct item indices
    
    # Generate predictions using the correct item indices
    scores = model.predict(user_index, np.array(item_indices))
    
    # Map scores to item IDs
    reverse_item_map = {v: k for k, v in dataset.mapping()[2].items()}
    scored_items = [(score, reverse_item_map[i]) for score, i in zip(scores, item_indices)]
    
    # Proceed with filtering scores for latest items and ranking them as before
    latest_transactions = transactions_df[transactions_df['reviewerID'] == user_id]\
                            .sort_values('reviewDate', ascending=False)\
                            .head(10)
    latest_items = set(latest_transactions['ASIN'])
    latest_scores = [score for score in scored_items if score[1] in latest_items]
    latest_scores.sort(reverse=True, key=lambda x: x[0])  # Sort based on scores
    top_item_ids = [item for _, item in latest_scores[:n]]

    return top_item_ids


In [11]:
uid = 'Laura_'

# Filter the DataFrame for the given reviewerID and select relevant columns
print(f"Top 10 Latest Reviews by {uid}")
reviewer_reviews_df = reviews[reviews['reviewerID'] == uid][['ASIN','ProductName','reviewRating','reviewDate']]
reviewer_reviews_df.sort_values(by='reviewDate', ascending=False).head(10)

Top 10 Latest Reviews by Laura_


,ASIN,ProductName,reviewRating,reviewDate
223577,0593321200,tomorrow_and_tomorrow_and_tomorrow:_a_novel,5.0,2024-02-14
243390,b0cl49tw5w,stanley_classic_legendary_camp_mug,5.0,2024-02-12
206156,b0b52zy9j9,otostar_throw_pillow_insert_12_x_20_cushion_in...,5.0,2024-01-27
255791,b0c9pdfttl,byybuo_android_13_tablet101_inch_tablet_with_c...,5.0,2024-01-25
280203,b0ckp5c4y8,"atumtek_55""_selfie_stick_tripod_allinone_exten...",4.0,2024-01-21
205446,b07kq2j3vr,elegant_comfort_luxury_ultrasoft_2piece_pillow...,5.0,2024-01-17
304814,b0c7jpypmv,levi's_women's_726_high_rise_flare_jeans_also_...,5.0,2024-01-05
275628,b0bmzzv6r3,mattel_disney_wish_singing_asha_of_rosas_fashi...,5.0,2024-01-05
314457,b09pzsljz6,oqq_women's_3_piece_medium_support_crop_top_se...,5.0,2024-01-05
31215,b09jkyy9yj,lego_speed_champions_1970_ferrari_512_m_76906_...,5.0,2024-01-04


In [12]:
top_recommendations = get_top_n_recommendations(model, dataset, user_id=uid, n=10)
recommended_products_df = reviews[reviews['ASIN'].isin(top_recommendations)].drop_duplicates(subset=['ASIN'])
recommended_products_names = recommended_products_df[['ASIN', 'ProductName', 'reviewRating']].drop_duplicates()
print(f'Top-N Recommendations for {uid} based on All Transactions')
recommended_products_names

Top-N Recommendations for Laura_ based on All Transactions


,ASIN,ProductName,reviewRating
17732,b07rshbsht,duracell_optimum_aa_batteries_with_power_boost...,5.0
62998,b004guu7zs,jayden_4in1_mini_convertible_crib_and_changer_...,5.0
66314,b08l4s8cbv,homeideas_sage_green_sheer_curtains_52_x_63_in...,5.0
109667,b0cn2vlq4x,utag_hidden_gps_tracker_for_kids_2_pack_blueto...,5.0
149872,b0cmqrhzqn,etronik_weekender_bag_for_men_women_travel_duf...,5.0
164644,b0b99h35tb,jeoyoo_wall_mirror_full_length_cheap_over_/_on...,5.0
253616,b085sypmlv,firstime_cor_westbrook_farmhouse_cottage_galva...,5.0
272298,b00iowbkqo,wiggle_car_ride_on_toy__no_batteries_gears_or_...,5.0
286205,b091sy2byw,bellsal_silverware_organizer_kitchen_drawer_or...,5.0
292315,b081lj5dp6,bariatric_fusion_high_adek_multivitamin_capsul...,5.0


In [13]:
top_recommendations_latest = get_top_n_recommendations_with_latest_transactions(
    model=model,
    dataset=dataset,
    user_id=uid,
    transactions_df=reviews,
    n=10
)

latest_recommended_products_df = reviews[reviews['ASIN'].isin(top_recommendations_latest)].drop_duplicates(subset=['ASIN'])
latest_recommended_products_names = latest_recommended_products_df[['ASIN', 'ProductName', 'reviewRating']].drop_duplicates()
print(f'Top-N Recommendations for {uid} Based on Latest Reviews')
latest_recommended_products_names

Top-N Recommendations for Laura_ Based on Latest Reviews


,ASIN,ProductName,reviewRating
31203,b09jkyy9yj,lego_speed_champions_1970_ferrari_512_m_76906_...,5.0
205438,b07kq2j3vr,elegant_comfort_luxury_ultrasoft_2piece_pillow...,5.0
206148,b0b52zy9j9,otostar_throw_pillow_insert_12_x_20_cushion_in...,5.0
223573,0593321200,tomorrow_and_tomorrow_and_tomorrow:_a_novel,5.0
243381,b0cl49tw5w,stanley_classic_legendary_camp_mug,5.0
255781,b0c9pdfttl,byybuo_android_13_tablet101_inch_tablet_with_c...,5.0
275618,b0bmzzv6r3,mattel_disney_wish_singing_asha_of_rosas_fashi...,5.0
280195,b0ckp5c4y8,"atumtek_55""_selfie_stick_tripod_allinone_exten...",5.0
304804,b0c7jpypmv,levi's_women's_726_high_rise_flare_jeans_also_...,5.0
314449,b09pzsljz6,oqq_women's_3_piece_medium_support_crop_top_se...,5.0


### Model Evaluation

- **ROC AUC metric:** the probability that a randomly chosen positive example has a higher score than a randomly chosen negative example. A perfect score is 1.0.
- **Precision at k**: the fraction of known positives in the first k positions of the ranked list of results. A perfect score is 1.0.
- **Recall at k**: the number of positive items in the first k positions of the ranked list of results divided by the number of positive items in the test period. A perfect score is 1.0.

In [15]:
from tqdm import tqdm

k_values = range(5, 11)
auc_scores = []
precision_scores = []
recall_scores = []

# Wrapping k_values with tqdm for progress bar
for k in tqdm(k_values, desc="Processing"):
    auc = auc_score(model, Test).mean()
    precision = precision_at_k(model, Test, k=k).mean()
    recall = recall_at_k(model, Test, k=k).mean()
    
    auc_scores.append(auc)
    precision_scores.append(precision)
    recall_scores.append(recall)

print("Test Results:")
for k, auc, precision, recall in zip(k_values, auc_scores, precision_scores, recall_scores):
    print(f'K = {k}: AUC score: {auc}, Precision: {precision}, Recall: {recall}')

Processing: 100%|██████████| 6/6 [17:29<00:00, 174.92s/it]

Test Results:
K = 5: AUC score: 0.5017999410629272, Precision: 3.720993117894977e-05, Recall: 0.00016913605304106624
K = 6: AUC score: 0.5017999410629272, Precision: 3.6646146327257156e-05, Recall: 0.00020296326364927948
K = 7: AUC score: 0.5017999410629272, Precision: 3.141098204650916e-05, Recall: 0.00020296326364927948
K = 8: AUC score: 0.5017999410629272, Precision: 2.959880839625839e-05, Recall: 0.00021987686895338611
K = 9: AUC score: 0.5017999410629272, Precision: 2.631005372677464e-05, Recall: 0.00021987686895338611
K = 10: AUC score: 0.5017999410629272, Precision: 2.8753129299730062e-05, Recall: 0.000270617684865706


In [17]:
k_values = range(5, 11)
auc_scores = []
precision_scores = []
recall_scores = []

# Wrapping k_values with tqdm for progress bar
for k in tqdm(k_values, desc="Processing Train"):
    auc = auc_score(model, Train).mean()
    precision = precision_at_k(model, Train, k=k).mean()
    recall = recall_at_k(model, Train, k=k).mean()
    
    auc_scores.append(auc)
    precision_scores.append(precision)
    recall_scores.append(recall)

print("Train Results:")
for k, auc, precision, recall in zip(k_values, auc_scores, precision_scores, recall_scores):
    print(f'K = {k}: AUC score: {auc}, Precision: {precision}, Recall: {recall}')

Processing Train: 100%|██████████| 6/6 [1:05:42<00:00, 657.02s/it]

Train Results:
K = 5: AUC score: 0.5231432914733887, Precision: 3.082041075685993e-05, Recall: 0.00012917378633296018
K = 6: AUC score: 0.5231432914733887, Precision: 3.1726893212180585e-05, Recall: 0.00014654803829587294
K = 7: AUC score: 0.5231432914733887, Precision: 3.237438068026677e-05, Recall: 0.00018054113996244142
K = 8: AUC score: 0.5231432914733887, Precision: 3.285999991931021e-05, Recall: 0.00021377883936975282
K = 9: AUC score: 0.5231432914733887, Precision: 3.323769851704128e-05, Recall: 0.0002500381478140925
K = 10: AUC score: 0.5231432914733887, Precision: 3.35398581228219e-05, Recall: 0.0002810096404436326


### Interpretation of Results

- The AUC score is consistently high (0.9986754059791565) across all K values, indicating excellent overall model performance in distinguishing between positive and negative interactions.

- Precision at K measures the proportion of recommended items in the top-K set that are relevant. Precision decreases as K increases in your results, which is expected. When you recommend more items (increase K), the likelihood of including irrelevant items increases, thus lowering precision. However, a precision above 0.08 even at K=10 is quite good, suggesting that the top recommended items are largely relevant.

- Recall at K measures the proportion of relevant items that are successfully recommended in the top-K set. Your results show increasing recall with K, which is typical. As you recommend more items, you're more likely to cover a higher fraction of the relevant items, hence the recall increases. A recall of 0.8549128186215507 at K=10 indicates that about 85% of the relevant items are captured in the top 10 recommendations.